# 1.load data

In [116]:
import numpy as np
import pandas as pd
from collections import Counter

# read file
data = pd.read_csv('pop_music_chords.csv')

In [117]:
data.head()

,X...,X.....,X.......,X......1,X......2,X....,X....1,X..,X....2,X.....1
0,F,Am,C,C,C,Dm7,Am,Em,C,C
1,Em7,Em,Em,G/B,C/B,Am,F,D,Am,Em7
2,Dm,F,Am,Am,Am7,Dm7,G,C,F,Am7
3,C,G,F,Em,C/G,Am,C,G/B,G,Em/G
4,F,C,C,Fmaj7,C/F,F,Am,Am7,Em,C


# 2.train test split

In [118]:
data.shape

(47, 10)

In [119]:
n = 2
progression = []
for i in range(0,data.shape[1]):
    chords=data.iloc[:,i].values
    chords=chords[~pd.isna(chords)]
    progression.append(chords)

In [120]:
####split into test and train
from sklearn.model_selection import train_test_split
train, test = train_test_split(progression, test_size=0.2, random_state=42)

In [121]:
total_train = train[0]
for i in range(1,len(train)):
    chord = train[i]
    total_train = np.hstack((total_train,chord))

In [198]:
for i in range(0,8):
    print(len(progression[i]))

27
46
33
40
41
24
47
43


In [122]:
total_train

array(['Dm7', 'Am', 'Dm7', 'Am', 'F', 'C', 'Dm7', 'G', 'Dm7', 'Am', 'Dm7',
       'Am', 'F', 'C', 'Dm7', 'G', 'F', 'G', 'Em', 'Am', 'Dm7', 'G', 'C',
       'C7', 'F', 'Em7', 'Dm', 'C', 'F', 'Em7', 'Dm', 'C', 'C7', 'F', 'G',
       'Em', 'Am', 'Dm7', 'G', 'C', 'C7', 'F', 'G', 'Em', 'Am', 'Dm7',
       'G', 'F', 'Em', 'Dm', 'C', 'Em', 'D', 'C', 'G/B', 'Am7', 'B7',
       'Em', 'B7', 'Em', 'D', 'C', 'G/B', 'Am7', 'B7', 'Em', 'Em', 'D',
       'C', 'G/B', 'Am7', 'B7', 'Em', 'Em', 'D', 'C', 'G/B', 'Am7', 'B7',
       'Em', 'Em', 'D', 'C', 'G/B', 'Am7', 'B7', 'Em', 'Em', 'D', 'C',
       'G/B', 'Am7', 'B7', 'Em', 'C', 'Em', 'Am', 'F', 'C', 'Em', 'Am',
       'F', 'C', 'Em', 'Am', 'F', 'Dm7', 'G', 'Dm7', 'G', 'G', 'C', 'Em',
       'Am', 'Fm', 'G', 'C', 'Em', 'Am', 'Fm', 'G', 'C', 'Em', 'Am', 'Fm',
       'G', 'C', 'C', 'Em7', 'Am7', 'Em/G', 'C', 'Em7', 'F', 'G', 'C',
       'Em7', 'Am7', 'Em/G', 'C', 'Em7', 'F', 'G', 'Gm', 'F', 'Dm7', 'Fm',
       'Gsus4', 'G', 'C', 'G/B', 'Am', 'G', 'F', 'C

In [123]:
ngrams = zip(*[total_train[i:] for i in range(n)])
bigrams = [" ".join(ngram) for ngram in ngrams]

In [124]:
bigrams

['Dm7 Am',
 'Am Dm7',
 'Dm7 Am',
 'Am F',
 'F C',
 'C Dm7',
 'Dm7 G',
 'G Dm7',
 'Dm7 Am',
 'Am Dm7',
 'Dm7 Am',
 'Am F',
 'F C',
 'C Dm7',
 'Dm7 G',
 'G F',
 'F G',
 'G Em',
 'Em Am',
 'Am Dm7',
 'Dm7 G',
 'G C',
 'C C7',
 'C7 F',
 'F Em7',
 'Em7 Dm',
 'Dm C',
 'C F',
 'F Em7',
 'Em7 Dm',
 'Dm C',
 'C C7',
 'C7 F',
 'F G',
 'G Em',
 'Em Am',
 'Am Dm7',
 'Dm7 G',
 'G C',
 'C C7',
 'C7 F',
 'F G',
 'G Em',
 'Em Am',
 'Am Dm7',
 'Dm7 G',
 'G F',
 'F Em',
 'Em Dm',
 'Dm C',
 'C Em',
 'Em D',
 'D C',
 'C G/B',
 'G/B Am7',
 'Am7 B7',
 'B7 Em',
 'Em B7',
 'B7 Em',
 'Em D',
 'D C',
 'C G/B',
 'G/B Am7',
 'Am7 B7',
 'B7 Em',
 'Em Em',
 'Em D',
 'D C',
 'C G/B',
 'G/B Am7',
 'Am7 B7',
 'B7 Em',
 'Em Em',
 'Em D',
 'D C',
 'C G/B',
 'G/B Am7',
 'Am7 B7',
 'B7 Em',
 'Em Em',
 'Em D',
 'D C',
 'C G/B',
 'G/B Am7',
 'Am7 B7',
 'B7 Em',
 'Em Em',
 'Em D',
 'D C',
 'C G/B',
 'G/B Am7',
 'Am7 B7',
 'B7 Em',
 'Em C',
 'C Em',
 'Em Am',
 'Am F',
 'F C',
 'C Em',
 'Em Am',
 'Am F',
 'F C',
 'C Em',
 'Em 

# 3.Predict next state with Markov chain

In [125]:
def predict_next_state(chord:str, data:list=bigrams):
    """Predict next chord based on current state."""
    # create list of bigrams which stats with current chord
    bigrams_with_current_chord = [bigram for bigram in bigrams if bigram.split(' ')[0]==chord]
    # count appearance of each bigram
    count_appearance = dict(Counter(bigrams_with_current_chord))
    # convert apperance into probabilities
    for ngram in count_appearance.keys():
        count_appearance[ngram] = count_appearance[ngram]/len(bigrams_with_current_chord)
    # create list of possible options for the next chord
    options = [key.split(' ')[1] for key in count_appearance.keys()]
    # create  list of probability distribution
    probabilities = list(count_appearance.values())
    # return random prediction
    return np.random.choice(options, p=probabilities)

In [204]:
predict_next_state("C",bigrams)

'G/B'

# 4.Generate sequence

In [61]:
def generate_sequence(chord:str=None, data:list=bigrams, length:int=30):
    """Generate sequence of defined length."""
    # create list to store future chords
    chords = []
    for n in range(length):
        # append next chord for the list
        chords.append(predict_next_state(chord, bigrams))
        # use last chord in sequence to predict next chord
        chord = chords[-1]
    return chords

In [62]:
generate_sequence('C')

['F',
 'Em7',
 'Dm',
 'C',
 'C7',
 'F',
 'Em',
 'Dm',
 'C',
 'F',
 'Em7',
 'Dm',
 'C',
 'C7',
 'F',
 'G',
 'Em',
 'Am',
 'Dm7',
 'G',
 'F',
 'Em7',
 'Dm',
 'C',
 'F',
 'G',
 'Em',
 'Am',
 'Dm7',
 'G']